# Evolution of sequence, structure and dynamics with *Evol* and *SignDy*

This tutorial has two parts, focusing on two related parts of *ProDy* for studying evolution: 

1. The sequence sub-package [*Evol*](http://prody.csb.pitt.edu/evol/) is for fetching, parsing and refining multiple sequence alignments (MSAs), and calculating residue-level properties such as conservation and coevolution as well as sequence-level properties such as percentage identity.


2. The signature dynamics module [*SignDy*](http://prody.csb.pitt.edu/signdy/) calculates ENM normal modes for ensembles of related protein structures and evaluates the conservation and differentiation of signature dynamics across families and subfamilies. It also allows classification of ensemble/family members based upon their dynamics, allowing the evolution of protein dynamics to be compared with the evolution of sequence and structure.

We first make the required imports:

In [ ]:
from prody import *
from pylab import *
%matplotlib inline
confProDy(auto_show=False)

We also configure *ProDy* to put all the PDB files in a particular folder seeing as there are so many of them. 

In [ ]:
pathPDBFolder('./pdbs/')

## 1. Sequence evolution with *Evol*

### Fetching, parsing and refining MSAs from Pfam

The protein families database [Pfam](https://pfam.xfam.org/) provides multiple sequence alignments of related protein domains, which we are often used as starting points for sequence evolution analyses. We can fetch such MSAs using the function [`fetchPfamMSA`](http://prody.csb.pitt.edu/manual/reference/database/pfam.html?highlight=fetchpfammsa#prody.database.pfam.fetchPfamMSA) as follows:

In [ ]:
filename = fetchPfamMSA('PF00074')
filename

We can then parse the MSA into *ProDy* using the [`parseMSA`](http://prody.csb.pitt.edu/manual/reference/sequence/msafile.html?highlight=parsemsa#prody.sequence.msafile.parseMSA) function, which can handle various types of MSA files including Stockholm, SELEX, CLUSTAL, PIR and FASTA formats. 

In [ ]:
msa = parseMSA(filename)
msa

This alignment can be indexed to extract individual sequences (rows) and residue positions (columns):

In [ ]:
msa[:10,:10]

In [ ]:
seq0 = msa[0]
seq0

In [ ]:
str(seq0)

This alignment contains many redundant sequences as well as lots of rows and columns with large numbers of gaps. Therefore, we refine it using [`refineMSA`](http://prody.csb.pitt.edu/manual/reference/sequence/msa.html?highlight=refinemsa#prody.sequence.msa.refineMSA), which we can do based on the sequence of RNAS1_BOVIN:

In [ ]:
msa_refined = refineMSA(msa, label='RNAS1_BOVIN', rowocc=0.8, seqid=0.98)
msa_refined

### Measuring sequence conservation with Shannon entropy

We calculate use [`calcShannonEntropy`](http://prody.csb.pitt.edu/manual/reference/sequence/analysis.html?highlight=shannonentropy#prody.sequence.analysis.calcShannonEntropy) to calculate the entropy of the refined MSA, which is a measure of sequence variability.

Shannon's entropy measures the degree of uncertainty that exists in a system. In the case of multiple sequence alignments, the Shannon entropy of each protein site (column) can be computed according to: 

$$H(p_1, p_2, \ldots, p_n) = -\sum_{i=1}^n p_i \log_2 p_i $$

where $p_i$ is the frequency of amino acid $i$ in that site. If a column is completely conserved then Shannon entropy is 0. The maximum variability, where each amino acid occurs with frequency 1/20, yields an entropy of 4.32

In [ ]:
entropy = calcShannonEntropy(msa_refined)

We can also show the Shannon entropy on a bar chart:

In [ ]:
showShannonEntropy(msa_refined);

### Comparisons of sequence evolution and structural dynamics

Next, we obtain residue fluctuations or mobility for a protein member of the above family using the GNM. 

We will use chain B of PDB structure 2W5I, which corresponds to our reference sequence RNAS1_BOVIN.

In [ ]:
ag = parsePDB('2W5I', chain='B')
ag

The next step is to select the corresponding residues from the AtomGroup to match the sequence alignment. We can identify these using [`alignSequenceToMSA`](http://prody.csb.pitt.edu/manual/reference/sequence/analysis.html?highlight=alignsequencetomsa#prody.sequence.analysis.alignSequenceToMSA). We give it the Calpha atoms only so the residue numbers aren't repeated.

In [ ]:
aln, idx_1, idx_2 = alignSequenceToMSA(ag.ca, msa_refined, label='RNAS1_BOVIN')
showAlignment(aln, indices=[idx_1, idx_2])

We see that there are extra residues in the PDB sequence compared to the reference sequence so we identify their residue numbers to make a selection.

In [ ]:
print(ag.ca.getResnums())

They are numbered from 1 to 124, two residues are missing from the beginning, and three residues are missing from the end, so we select residues 3 to 121. This now makes the two sequences match.

In [ ]:
chB = ag.select('resid 3 to 121')
chB

In [ ]:
print(msa_refined['RNAS1_BOVIN'])
print(chB.ca.getSequence())

We perform GNM analysis as follows:

In [ ]:
gnm = GNM('2W5I')
gnm.buildKirchhoff(chB.ca)
gnm.calcModes(n_modes=None)  # calculate all modes

We can then visually compare the behaviour at the individual residue level as follows:

In [ ]:
mobility = calcSqFlucts(gnm)

In [ ]:
figure(figsize=(13,6))

# plot entropy as grey bars
bar(chB.ca.getResnums(), entropy, width=1.2, color='grey', label='entropy');

# rescale mobility
mobility = mobility*(max(entropy)/max(mobility))

# plot mobility as a blue line
showAtomicLines(mobility, atoms=chB.ca, color='b', linewidth=2, label='mobility');

legend()

### Coevolution Calculation

In addition to the conservation/variation of individual positions, we can also calculate the coevolution between positions due to correlated mutations.

One simple and common method for this is to compute the mutual information between the columns in the MSA:

In [ ]:
mutinfo = buildMutinfoMatrix(msa_refined)

In [ ]:
showMutinfoMatrix(msa_refined, cmap='inferno');
title(None);

We can improve this with the widely used average product correction:

In [ ]:
mi_apc = applyMutinfoCorr(mutinfo)

In [ ]:
showMatrix(mi_apc, cmap='inferno');

We can change the colour scale normalisation to eliminate the effect of the diagonal. However, the mutual information matrix is still pretty noisy.

In [ ]:
showMatrix(mi_apc, cmap='inferno', norm=Normalize(0, 0.5));

Therefore, more sophisticated analyses have also been developed including the Direct Information (DI; also known as direct coupling analysis (DCA), which is very successful for **contact prediction**. This method can also be used in *ProDy* as follows:

In [ ]:
di = buildDirectInfoMatrix(msa_refined)

In [ ]:
showDirectInfoMatrix(msa_refined, cmap='inferno');
title(None);

If we compare the brighter regions on this map to the contact matrix then we see that they indeed match pretty well:

In [ ]:
showContactMap(gnm, origin='lower', cmap='Greys');

We can also apply a rank-ordering to the DI and corrected MI matrix entries, which helps identify the strongest signals:

In [ ]:
di_rank_row, di_rank_col, di_zscore_sort = calcRankorder(di, zscore=True)
print('row:   ', di_rank_row[:5])
print('column:', di_rank_col[:5])

In [ ]:
mi_rank_row, mi_rank_col, mi_zscore_sort = calcRankorder(mi_apc, zscore=True)
print('row:   ', mi_rank_row[:5])
print('column:', mi_rank_col[:5])

## 2. Signature Dynamics analysis with *SignDy*

This tutorial describes how to calculate signature dynamics for a family of proteins with similar structures using Elastic Network Models (ENMs). This method (also called ensemble normal mode analysis) creates  an ensemble of aligned structures and calculates statistics such as means and  standard deviations on various dynamic properties including mode profiles, mean square fluctuations and cross-correlation matrices. It also includes tools for classifying family members based on their sequence, structure and dynamics.

The theory and usage of this toolkit is described in our recent paper:

Zhang S, Li H, Krieger J, Bahar I. 
Shared signature dynamics tempered by local fluctuations enables fold adaptability and specificity. *Mol. Biol. Evol.* **2019** 36(9):2053–2068

In this tutorial, we will have a quick walk-through on the *SignDy* calculations and functions using the example of type-I 
periplasmic binding protein (PBP-I) domains. The data is collected using the Dali server (http://ekhidna2.biocenter.helsinki.fi/dali/).

Holm L, Rosenström P.
Dali server: conservation mapping in 3D.
*Nucleic Acids Res.* **2010** 10(38):W545-9

In addition to the previous imports, we also import time so that we can use the `sleep` function to reduce the load on the Dali server.

In [ ]:
import time

### Overview

The first step in signature dynamics analysis is to collect a set of related protein structures and build a [`PDBEnsemble`](http://prody.csb.pitt.edu/manual/reference/ensemble/pdbensemble.html?highlight=pdbensemble#module-prody.ensemble.pdbensemble). This can be achieved by multiple routes: a query search of the PDB using [`blastPDB`](http://prody.csb.pitt.edu/manual/reference/proteins/blastpdb.html?highlight=blastpdb#prody.proteins.blastpdb.blastPDB) or Dali, extraction of PDB IDs from the Pfam database (as above) or the CATH database, or input of a pre-defined list. 

We demonstrate the Dali method here in the first part of the tutorial. The usage of CATH methods is described in the [website tutorial](http://prody.csb.pitt.edu/tutorials/signdy_tutorial/cath.html) and the function [`blastPDB`](http://prody.csb.pitt.edu/manual/reference/proteins/blastpdb.html?highlight=blastpdb#prody.proteins.blastpdb.blastPDB) is described in the [Structure Analysis Tutorial](http://prody.csb.pitt.edu/tutorials/structure_analysis/).

We apply these methods to the PBP-I domains, a group of protein structures originally found in bacteria for transport of solutes across the periplasmic space and later seen in various eukaryotic receptors including ionotropic and metabotropic glutamate receptors. We use the N-terminal domain of AMPA receptor subunit GluA2 (gene name GRIA2; <https://www.uniprot.org/uniprot/P42262>) as a query.

The second step is then to calculate ENM normal modes for all members of the [`PDBEnsemble`](http://prody.csb.pitt.edu/manual/reference/ensemble/pdbensemble.html?highlight=pdbensemble#module-prody.ensemble.pdbensemble), creating a [`ModeEnsemble`](http://prody.csb.pitt.edu/manual/reference/dynamics/signature.html?highlight=modeensemble#prody.dynamics.signature.ModeEnsemble). We usually use the GNM for this as will be shown here, but the ANM can be used too. 

The third step is then to analyse conserved and divergent behaviours to identify signature dynamics of the whole family or individual subfamilies. This is aided calculations of overlaps and distances between the mode spectra (step 4), which can be used to create phylogenetic trees that can be compared to sequence and structural conservation and divergence. 

### Step 1: Prepare Ensemble (using Dali)

First we use the function [`searchDali`](http://prody.csb.pitt.edu/manual/reference/database/dali.html?highlight=searchdali#prody.database.dali.searchDali) to search the PDB with Dali, which returns a [`DaliRecord`](http://prody.csb.pitt.edu/manual/reference/database/dali.html?highlight=dalirecord#prody.database.dali.DaliRecord) object that contains a list of PDB IDs and their corresponding mappings to the reference structure. 

In [ ]:
dali_rec = searchDali('3H5V','A')
dali_rec

The Dali search often remains in the queue longer than the timeout time. We therefore have a `fetch` method, which can be run later to fetch the data. We can run this in a loop with a wait of a couple of minutes in between fetches to make sure we get the result.

In [ ]:
while not dali_rec.isSuccess:
    dali_rec.fetch()
    time.sleep(120)
    
dali_rec

Next, we get the lists of PDB IDs and mappings from *dali_rec*, and parse the *pdb_ids* to get a list of [`AtomGroup`](http://prody.csb.pitt.edu/manual/reference/atomic/atomgroup.html?highlight=atomgroup#module-prody.atomic.atomgroup) instances:

In [ ]:
pdb_ids = dali_rec.filter(cutoff_len=0.7, cutoff_rmsd=1.0, cutoff_Z=30)

In [ ]:
mappings = dali_rec.getMappings()

In [ ]:
ags = parsePDB(pdb_ids, subset='ca')
len(ags)

Then we provide *ags* together with *mappings* to [`buildPDBEnsemble`](http://prody.csb.pitt.edu/manual/reference/ensemble/functions.html?highlight=buildpdbensemble#prody.ensemble.functions.buildPDBEnsemble). We set the keyword argument ``seqid=20`` to account for the low sequence identity between some of the structures.

In [ ]:
dali_ens = buildPDBEnsemble(ags, mapping=mappings, seqid=20, labels=pdb_ids)
dali_ens

Finally, we save the ensemble for later processing:

In [ ]:
saveEnsemble(dali_ens, 'PBP-I')

### Step 2: Mode ensemble

For this analysis we'll build a [`ModeEnsemble`](http://prody.csb.pitt.edu/manual/reference/dynamics/signature.html?highlight=modeensemble#prody.dynamics.signature.ModeEnsemble) by calculating normal modes for each member of the [`PDBEnsemble`](http://prody.csb.pitt.edu/manual/reference/ensemble/pdbensemble.html?highlight=pdbensemble#module-prody.ensemble.pdbensemble). 

You can load a PDB ensemble at this stage if you already have one. We demonstrate this for the one we just saved.

In [ ]:
dali_ens = loadEnsemble('PBP-I.ens.npz')

Then we calculate [`GNM`](http://prody.csb.pitt.edu/manual/reference/dynamics/gnm.html?highlight=gnm#prody.dynamics.gnm.GNM) modes for each member of the ensemble using [`calcEnsembleENMs`](http://prody.csb.pitt.edu/manual/reference/dynamics/signature.html?highlight=calcensembleenms#prody.dynamics.signature.calcEnsembleENMs). There are options to select the *model* (`GNM` by default) and the way of considering non-aligned residues by setting the *trim* option (default is `reduceModel`, which treats them as environment).

In [ ]:
gnms = calcEnsembleENMs(dali_ens, model='GNM', trim='reduce')
gnms

We can save the mode ensemble as follows:

In [ ]:
saveModeEnsemble(gnms, 'PBP-I')

We can also load in a previously saved mode ensemble such as the one we saved above:

In [ ]:
gnms = loadModeEnsemble('PBP-I.modeens.npz')

### Slicing and Indexing Mode Ensembles

We can index the [`ModeEnsemble`](http://prody.csb.pitt.edu/manual/reference/dynamics/signature.html?highlight=modeensemble#prody.dynamics.signature.ModeEnsemble) object in two different dimensions. The first dimension corresponds to ensemble members as shown below for extracting the mode set for the first member (numbered 0).

In [ ]:
gnms[0]

The second dimension corresponds to particular modes of all ensemble members as shown below for extracting the first mode (numbered 0). The colon means we select everything from the first dimension.

In [ ]:
gnms[:,0]

We can also slice out ranges of members and modes and index them both at the same time. E.g. to get the five members from 5 up to but not including 10 (5, 6, 7, 8, 9), and the two modes from 2 up to but not including 4 (modes with indices 2 and 3 in the reference), we'd use the following code.

In [ ]:
gnms[5:10,2:4]

We can also use indexing to extract individual modes from individual members, e.g. 

In [ ]:
gnms[5,2]

Remember that we usually talk about modes counting from 1 so this is "Mode 3" or "the 3rd global mode" in conversation but Python counts from 0 so it has index 2. Likewise this is the "6th member" of the ensemble but has index 5.

### Step 3: Signature dynamics

Signatures are calculated as the mean and standard deviation of various properties 
such as mode shapes and mean square fluctations.

For example, we can show the average and standard deviation of the shape of the first 
mode (second index 0). The first index of the mode ensemble is over conformations.

In [ ]:
showSignatureMode(gnms[:, 0]);

We can also show such results for properties involving multiple modes such as the mean square fluctuations from the first 5 modes or the cross-correlations from the first 20.

In [ ]:
showSignatureSqFlucts(gnms[:, :5]);

In [ ]:
showSignatureCrossCorr(gnms[:, :20]);

We can also look at distributions over values across different members of the ensemble such as inverse eigenvalue. We can show a bar above this with individual members labelled 
like in 

Krieger J, Bahar I, Greger IH.
Structure, Dynamics, and Allosteric Potential of Ionotropic Glutamate Receptor N-Terminal Domains. *Biophys. J.* **2015** 109(6):1136-48.
   
In this automated version, the bar is coloured from white to dark red depending on how many structures have values at that point.

We can select particular members to highlight with arrows by putting their names and labels in a dictionary:

In [ ]:
highlights = {'3h5vA': 'GluA2','3o21C': 'GluA3',
              '3h6gA': 'GluK2', '3olzA': 'GluK3', 
              '5kc8A': 'GluD2'}

We plot the variance bar for the first five modes (showing a function of the inverse eigenvalues related to the resultant relative size of motion) above the inverse eigenvalue distributions for each of those modes. To arrange the plots like this, we use the [`GridSpec`](https://matplotlib.org/3.1.3/api/_as_gen/matplotlib.gridspec.GridSpec.html) function of Matplotlib.

In [ ]:
gs = GridSpec(ncols=1, nrows=2, height_ratios=[1, 10], hspace=0.15)

subplot(gs[0]);
showVarianceBar(gnms[:, :5], fraction=True, highlights=highlights);
xlabel('');

subplot(gs[1]);
showSignatureVariances(gnms[:, :5], fraction=True, bins=80, alpha=0.7);
xlabel('Fraction of inverse eigenvalue');

We can also extract the eigenvalues and eigenvectors directly from the mode ensemble and analyse them ourselves:

In [ ]:
eigvals = gnms.getEigvals()
eigvals

In [ ]:
eigvecs = gnms.getEigvecs()
eigvecs

These are stored in instances of the [`sdarray`](http://prody.csb.pitt.edu/manual/reference/dynamics/signature.html?highlight=sdarray#prody.dynamics.signature.sdarray) class that we designed specifically for signature dynamics analysis. It is an extension of the standard NumPy [`ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) but has additional attributes and some modified methods. The first axis is reserved for ensemble members and the mean, min, max and std are altered to average over this dimension rather than all dimensions.

We can look at the shape of these arrays and index them just like `ndarray` and `ModeEnsemble` objects. The eigenvalues are arranged in **eigvals** such that the first axis is the members and the second is the modes as in the mode ensemble.

In [ ]:
eigvals.shape

In [ ]:
eigvals[0:5,0:5]

The eigenvectors are arranged in **eigvecs** such that the first axis is over the members, and the remaining dimensions are as in other eigenvector arrays - the second is over atoms and the third is mode index. Each atom has a weight, which varies between members and is important in calculating the mean, std, etc. 

In [ ]:
eigvecs.shape

### Step 4: Spectral overlap and distance

Spectral overlap, also known as covariance overlap, measures the overlap between two covariance matrices, or the overlap of a subset of the modes (a mode spectrum). This can also be converted into a distance using its arccosine as will be shown below.

We can calculate a matrix of spectral overlaps (**so_matrix**) over any slice of the ModeEnsemble that is still a mode ensemble itself, e.g.

In [ ]:
so_matrix = calcEnsembleSpectralOverlaps(gnms[:, :1])

In [ ]:
figure(figsize=(8,8))
showMatrix(so_matrix);

We can also obtain a spectral distance matrix (**sd_matrix**) from [`calcEnsembleSpectralOverlaps`](http://prody.csb.pitt.edu/manual/reference/dynamics/signature.html?highlight=calcensemblespectraloverlaps#prody.dynamics.signature.calcEnsembleSpectralOverlaps) by giving it an additional argument:

In [ ]:
sd_matrix = calcEnsembleSpectralOverlaps(gnms[:, :1], distance=True)
figure(figsize=(8,8)); showMatrix(sd_matrix);

We can then use this distance to calculate a tree. The labels from the mode ensemble as used as names for the leaves of the tree and are stored in their own variable/object for later use.

In [ ]:
labels = dali_ens.getLabels()
so_tree = calcTree(names=labels, distance_matrix=sd_matrix, method='upgma')

We can show this tree using the function [`showTree`](http://prody.csb.pitt.edu/manual/reference/dynamics/plotting.html?highlight=showtree#prody.dynamics.plotting.showTree):

In [ ]:
showTree(so_tree);

We can also use this tree to reorder the **so_matrix** and obtain indices for reordering other objects:

In [ ]:
reordered_so, new_so_indices = reorderMatrix(names=labels, matrix=so_matrix, tree=so_tree)

In [ ]:
figure(figsize=(8,8))
showMatrix(reordered_so, ticklabels=new_so_indices);

As in the tree, we see 2-3 clusters with some finer structure within them as in the tree. These correspond to different subtypes of iGluRs called AMPA receptors (subunit paralogues GluA1-4, top) and kainate receptors (subunit paralogues GluK1-5, bottom) based on their preferred agonists as well as delta receptors at the bottom (these are flipped relative to the tree).

To show the matrix in the same order as the tree, we can add the option `origin='upper'`:

In [ ]:
figure(figsize=(8,8))
showMatrix(reordered_so, ticklabels=new_so_indices, origin='upper');

We can also show the tree along the y-axis of the matrix as follows:

In [ ]:
figure(figsize=(11,8))
showMatrix(reordered_so, ticklabels=new_so_indices, origin='upper', 
           y_array=so_tree);

We can also use the resulting indices to reorder the `ModeEnsemble` and `PDBEnsemble`:

In [ ]:
so_reordered_ens = dali_ens[new_so_indices]
so_reordered_gnms = gnms[new_so_indices, :]

Lists can only be used for indexing arrays not lists so we need to perform a type conversion prior to indexing in order to reorder the labels:

In [ ]:
so_reordered_labels = np.array(labels)[new_so_indices]

### Comparing with sequence and structural distances

The sequence distance is given by the (normalized) Hamming distance, which is 
calculated by subtracting the percentage identity (fraction) from 1, and the 
structural distance is the RMSD. We can also calculate and show the matrices 
and trees for these from the PDB ensemble.

First we calculate the sequence distance matrix:

In [ ]:
seqid_matrix = buildSeqidMatrix(so_reordered_ens.getMSA())
seqdist_matrix = 1. - seqid_matrix

In [ ]:
figure(figsize=(8,8));
showMatrix(seqdist_matrix);

We can also construct a tree based on **seqdist_matrix** and use that to reorder it:

In [ ]:
seqdist_tree = calcTree(names=so_reordered_labels, distance_matrix=seqdist_matrix, method='upgma')
showTree(seqdist_tree);

We can reorder **seqdist_matrix** with **seqdist_tree** as we did above with **so_tree**:

In [ ]:
reordered_seqdist_seqdist, new_seqdist_indices = reorderMatrix(names=so_reordered_labels, 
                                                               matrix=seqdist_matrix, tree=seqdist_tree)
figure(figsize=(8,8));
showMatrix(reordered_seqdist_seqdist, ticklabels=new_seqdist_indices);

This shows us even clearer groups than the dynamic spectrum-based analysis. We see one subunit by itself at the bottom that is from a delta-type iGluR (GluD2), then two groups of kainate receptors (GluK5 and GluK2 with GluK3), and four groups of AMPARs (GluA1, GluA2, GluA4, and many structures from GluA3).

Similarily, once we obtain the RMSD matrix and tree using the [`getRMSDs`](http://prody.csb.pitt.edu/manual/reference/ensemble/pdbensemble.html?highlight=pdbensemble.getrmsds#prody.ensemble.pdbensemble.PDBEnsemble.getRMSDs) method of the `PDBEnsemble`, we 
can calculate the structure-based tree:

In [ ]:
rmsd_matrix = so_reordered_ens.getRMSDs(pairwise=True)
figure(figsize=(8,8)); showMatrix(rmsd_matrix);

In [ ]:
rmsd_tree = calcTree(names=so_reordered_labels, 
                     distance_matrix=rmsd_matrix, 
                     method='upgma')

It could be of interest to put all three trees constructed based on different 
distance metrics side by side and compare them. We can do this using the [`subplot`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.subplot.html) function from Matplotlib.

In [ ]:
figure(figsize=(20,8));
subplot(1, 3, 1);
showTree(seqdist_tree, format='plt');
title('Sequence');
subplot(1, 3, 2);
showTree(rmsd_tree, format='plt');
title('Structure');
subplot(1, 3, 3);
showTree(so_tree, format='plt');
title('Dynamics');

Likewise, we can place the matrices side-by-side after having them all reordered the same way. We'll reorder by seqdist in this example:

In [ ]:
reordered_rmsd_seqdist, new_seqdist_indices = reorderMatrix(names=so_reordered_labels, 
                                                            matrix=rmsd_matrix, tree=seqdist_tree)
reordered_sd_seqdist, new_seqdist_indices = reorderMatrix(names=so_reordered_labels, 
                                                          matrix=sd_matrix, tree=seqdist_tree)

In [ ]:
figure(figsize=(20,8));
subplot(1, 3, 1);
showMatrix(reordered_seqdist_seqdist, ticklabels=new_seqdist_indices, origin='upper');
title('Sequence');
subplot(1, 3, 2);
showMatrix(reordered_rmsd_seqdist, ticklabels=new_seqdist_indices, origin='upper');
title('Structure');
subplot(1, 3, 3);
showMatrix(reordered_sd_seqdist, ticklabels=new_seqdist_indices, origin='upper');
title('Dynamics');

This analysis is quite sensitive to how many modes are used. As the number of modes approaches the full number, 
the dynamic distance order approaches the RMSD order. With smaller numbers, we see finer distinctions and there is a point where the dynamic distances are more in line with the sequence distances, which we call the low-to-intermediate frequency regime. In the current case where we used just one global mode (with the lowest frequency), we see small spectral distances but some subfamily differentiation is still apparent.

The same analysis could also be performed with a larger ensemble by selecting lower sequence identity and Z-score cutoffs as we did in our paper.

Now we have finished this tutorial, we reset the default path to the PDB folder, so that we aren't surprised next time we download PDBs and can't find them:

In [ ]:
pathPDBFolder('')